In [1]:
# basic (built-in) Python packages
import numpy as np
from numpy.linalg import norm
import pandas as pd
import matplotlib.pyplot as plt
from copy import copy
import sys
sys.path.append('../')

# advanced (built-in) Python packages
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from scipy.optimize import minimize
from scipy import stats
from scipy.stats import norm
import tensorflow as tf
import keras

# my implemented Python functions and classes
from utils import *

# Deep Learning Library
import tensorflow as tf
import tensorflow.keras as keras
from keras.models import Model
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow.keras.applications.efficientnet import preprocess_input

In [2]:
K = 9
EPOCHS = 50

In [3]:
from keras.datasets import mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()

# change Xi from (28 x 28) into (28 x 28 x 3)
tmp = np.ones(train_X.shape + (3,))
tmp *= train_X.reshape(train_X.shape + (1,))
train_X = tmp
tmp = np.ones(test_X.shape + (3,))
tmp *= test_X.reshape(test_X.shape + (1,))
test_X = tmp

# onehot Yi
train_y_onehot = tf.keras.utils.to_categorical(train_y, K + 1)
test_y_onehot = tf.keras.utils.to_categorical(test_y, K + 1)
print(train_X.shape, test_X.shape)
print(train_y_onehot.shape, test_y_onehot.shape)

(60000, 28, 28, 3) (10000, 28, 28, 3)
(60000, 10) (10000, 10)


# Crowd Labeling 

- Use train crowd annotators (DLs) on the Testing Set and crowd-label the Training Set

In [4]:
base_path = f"/mnt/Crowdsourcing/codes/mnist_experiment/crowd models/"
inputs = tf.keras.layers.Input(shape=(28, 28, 3))

In [5]:
X0, X1, Y0, Y1 = train_test_split(test_X, test_y_onehot, test_size=0.1, random_state=0)
print(X0.shape, X1.shape, Y0.shape, Y1.shape)

(9000, 28, 28, 3) (1000, 28, 28, 3) (9000, 10) (1000, 10)


In [6]:
'''
Since input image size is (28 x 28), first upsample the image by factor of (8x8) to transform it to (224 x 224)
Connect the feature extraction and "classifier" layers to build the model.
'''
def final_model(inputs, feature_extractor_model, K):
    # Feature Extractor
    inputs = preprocess_input(inputs)
    resize = tf.keras.layers.UpSampling2D(size=(8, 8))(inputs)
    model_feature = feature_extractor_model(resize)
    # Classification
    '''
    Defines final dense layers and subsequent softmax layer for classification.
    '''
    x = tf.keras.layers.GlobalAveragePooling2D()(model_feature)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dense(K + 1, activation="softmax")(x)
    return x

In [7]:
def crowd_label(model, X):
    Y_prob = model.predict(X)
    Y_hat = np.argmax(Y_prob, axis=1)
    return Y_hat

## Model 1: MobileNet

In [8]:
model_name = "MobileNet"
model_path = base_path + f"{model_name}.keras"
Yhat_path = f"./crowd labels/{model_name}_y.npy"
callbacks_list = [ ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),
                   EarlyStopping(monitor='val_accuracy', patience=10, verbose=2, mode='auto'),]

In [9]:
feature_extractor = keras.applications.MobileNet(include_top=False, input_shape=(224, 224, 3), weights="imagenet")
feature_extractor.summary()

2024-09-28 12:53:40.280947: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-28 12:53:41.706313: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15401 MB memory:  -> device: 0, name: Tesla P100-SXM2-16GB, pci bus id: 0000:1d:00.0, compute capability: 6.0


17235968/17225924 [==============================] - 4s 0us/step
Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32

In [ ]:
# Model 1: MobileNet
feature_extractor = keras.applications.MobileNet(include_top=False, input_shape=(224, 224, 3), weights="imagenet")
output = final_model(inputs, feature_extractor, K) 
model = tf.keras.Model(inputs=inputs, outputs=output, name=model_name)
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(X0, Y0, validation_data = (X1, Y1), epochs=EPOCHS, batch_size=64, callbacks=callbacks_list,)

In [9]:
best_model = tf.keras.models.load_model(model_path)
Yhat = crowd_label(best_model, train_X)
print(f"Predict ACC: {(Yhat == train_y).sum() / len(Yhat)}")
np.save(Yhat_path, Yhat)

2024-09-29 02:15:20.026037: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-29 02:15:21.000357: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15401 MB memory:  -> device: 0, name: Tesla P100-SXM2-16GB, pci bus id: 0000:1d:00.0, compute capability: 6.0
2024-09-29 02:15:25.726687: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


Predict ACC: 0.9894


In [13]:
X_extractor = Model(inputs=best_model.input, outputs=best_model.get_layer('dense').output)
X = X_extractor.predict(train_X)
np.save("./feat_X.npy", X)

(60000, 256)

# Model 2: MobileNet-1

In [10]:
model_name = "MobileNet-1"
model_path = base_path + f"{model_name}.keras"
Yhat_path = f"./crowd labels/{model_name}_y.npy"
callbacks_list = [ ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),
                   EarlyStopping(monitor='val_accuracy', patience=10, verbose=2, mode='auto'),]

In [ ]:
# Model 2: MobileNet-1
feature_extractor = keras.applications.MobileNet(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
output = final_model(inputs, feature_extractor, K) 
model = tf.keras.Model(inputs=inputs, outputs=output, name=model_name)
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics = ['accuracy'])

In [10]:
history = model.fit(X0, Y0, validation_data = (X1, Y1), epochs=1, batch_size=64, callbacks=callbacks_list,)

2024-09-26 03:00:19.617628: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


141/141 [==============================] - ETA: 0s - loss: 0.1543 - accuracy: 0.9506
Epoch 1: val_accuracy improved from -inf to 0.96800, saving model to /mnt/Crowdsourcing/codes/mnist_experiment/crowd models/MobileNet-1.keras
141/141 [==============================] - 43s 266ms/step - loss: 0.1543 - accuracy: 0.9506 - val_loss: 0.1255 - val_accuracy: 0.9680


In [11]:
Yhat = crowd_label(model, train_X)
print(f"Predict ACC: {(Yhat == train_y).sum() / len(Yhat)}")
np.save(Yhat_path, Yhat)

Predict ACC: 0.9520833333333333


## Model 3: EfficientNet

In [11]:
model_name = "EfficientNet"
model_path = base_path + f"{model_name}.keras"
Yhat_path = f"./crowd labels/{model_name}_y.npy"
callbacks_list = [ ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),
                   EarlyStopping(monitor='val_accuracy', patience=10, verbose=2, mode='auto'),]

In [ ]:
# Model 3: EfficientNet
feature_extractor = keras.applications.EfficientNetB0(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
output = final_model(inputs, feature_extractor, K) 
model = tf.keras.Model(inputs=inputs, outputs=output, name=model_name)
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics = ['accuracy'])

In [12]:
history = model.fit(X0, Y0, validation_data = (X1, Y1), epochs=EPOCHS, batch_size=64, callbacks=callbacks_list,)

Epoch 1/50
141/141 [==============================] - ETA: 0s - loss: 0.1985 - accuracy: 0.9402
Epoch 1: val_accuracy improved from -inf to 0.96000, saving model to /mnt/Crowdsourcing/codes/mnist_experiment/crowd models/EfficientNet.keras
141/141 [==============================] - 68s 415ms/step - loss: 0.1985 - accuracy: 0.9402 - val_loss: 0.1510 - val_accuracy: 0.9600
Epoch 2/50
141/141 [==============================] - ETA: 0s - loss: 0.0469 - accuracy: 0.9849
Epoch 2: val_accuracy improved from 0.96000 to 0.96400, saving model to /mnt/Crowdsourcing/codes/mnist_experiment/crowd models/EfficientNet.keras
141/141 [==============================] - 56s 396ms/step - loss: 0.0469 - accuracy: 0.9849 - val_loss: 0.1538 - val_accuracy: 0.9640
Epoch 3/50
141/141 [==============================] - ETA: 0s - loss: 0.0412 - accuracy: 0.9884
Epoch 3: val_accuracy improved from 0.96400 to 0.98800, saving model to /mnt/Crowdsourcing/codes/mnist_experiment/crowd models/EfficientNet.keras
141/141 [

In [13]:
best_model = tf.keras.models.load_model(model_path)
Yhat = crowd_label(best_model, train_X)
print(f"Predict ACC: {(Yhat == train_y).sum() / len(Yhat)}")
# np.save(Yhat_path, Yhat)

Predict ACC: 0.9864333333333334


# Model 4: EfficientNet-1

In [ ]:
model_name = "EfficientNet-1"
model_path = base_path + f"{model_name}.keras"
Yhat_path = f"./crowd labels/{model_name}_y.npy"
callbacks_list = [ ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),
                   EarlyStopping(monitor='val_accuracy', patience=10, verbose=2, mode='auto'),]

In [15]:
# Model 4: EfficientNet-1
feature_extractor = keras.applications.EfficientNetB0(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
output = final_model(inputs, feature_extractor, K) 
model = tf.keras.Model(inputs=inputs, outputs=output, name=model_name)

model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics = ['accuracy'])

history = model.fit(X0, Y0, validation_data = (X1, Y1), epochs=1, batch_size=64, callbacks=callbacks_list,)

141/141 [==============================] - ETA: 0s - loss: 0.2023 - accuracy: 0.9394
Epoch 1: val_accuracy improved from -inf to 0.98400, saving model to /mnt/Crowdsourcing/codes/mnist_experiment/crowd models/EfficientNet-1.keras
141/141 [==============================] - 67s 410ms/step - loss: 0.2023 - accuracy: 0.9394 - val_loss: 0.0548 - val_accuracy: 0.9840


In [16]:
best_model = tf.keras.models.load_model(model_path)
Yhat = crowd_label(best_model, train_X)
np.save(Yhat_path, Yhat)

## Model 5: Xception

In [14]:
model_name = "Xception"
model_path = base_path + f"{model_name}.keras"
Yhat_path = f"./crowd labels/{model_name}_y.npy"
callbacks_list = [ ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),
                   EarlyStopping(monitor='val_accuracy', patience=10, verbose=2, mode='auto'),]

In [17]:
# Model 5: Xception
feature_extractor = keras.applications.Xception(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
output = final_model(inputs, feature_extractor, K) 
model = tf.keras.Model(inputs=inputs, outputs=output, name=model_name)
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics = ['accuracy'])

83697664/83683744 [==============================] - 120s 1us/step


In [18]:
history = model.fit(X0, Y0, validation_data = (X1, Y1), epochs=EPOCHS, batch_size=64, callbacks=callbacks_list,)

Epoch 1/50
141/141 [==============================] - ETA: 0s - loss: 0.1946 - accuracy: 0.9421
Epoch 1: val_accuracy improved from -inf to 0.82600, saving model to /mnt/Crowdsourcing/codes/mnist_experiment/crowd models/Xception.keras
141/141 [==============================] - 109s 725ms/step - loss: 0.1946 - accuracy: 0.9421 - val_loss: 1.0682 - val_accuracy: 0.8260
Epoch 2/50
141/141 [==============================] - ETA: 0s - loss: 0.0662 - accuracy: 0.9819
Epoch 2: val_accuracy did not improve from 0.82600
141/141 [==============================] - 95s 675ms/step - loss: 0.0662 - accuracy: 0.9819 - val_loss: 2.0840 - val_accuracy: 0.7650
Epoch 3/50
141/141 [==============================] - ETA: 0s - loss: 0.0510 - accuracy: 0.9870
Epoch 3: val_accuracy improved from 0.82600 to 0.91600, saving model to /mnt/Crowdsourcing/codes/mnist_experiment/crowd models/Xception.keras
141/141 [==============================] - 100s 709ms/step - loss: 0.0510 - accuracy: 0.9870 - val_loss: 0.2583

In [15]:
best_model = tf.keras.models.load_model(model_path)
Yhat = crowd_label(best_model, train_X)
print(f"Predict ACC: {(Yhat == train_y).sum() / len(Yhat)}")
# np.save(Yhat_path, Yhat)

Predict ACC: 0.9896


# Model 6: Xception-1

In [12]:
model_name = "Xception-1"
model_path = base_path + f"{model_name}.keras"
Yhat_path = f"./crowd labels/{model_name}_y.npy"
callbacks_list = [ ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),
                   EarlyStopping(monitor='val_accuracy', patience=10, verbose=2, mode='auto'),]

In [13]:
feature_extractor = keras.applications.Xception(include_top=False, input_shape=(224, 224, 3), weights="imagenet")
output = final_model(inputs, feature_extractor, K)
model = tf.keras.Model(inputs=inputs, outputs=output, name=model_name)
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics = ['accuracy'])

In [14]:
history = model.fit(X0, Y0, validation_data = (X1, Y1), epochs=1, batch_size=64, callbacks=callbacks_list)

Epoch 1/5
141/141 [==============================] - ETA: 0s - loss: 0.2219 - accuracy: 0.9361
Epoch 1: val_accuracy improved from -inf to 0.98800, saving model to /mnt/Crowdsourcing/codes/mnist_experiment/crowd models/Xception-5.keras
141/141 [==============================] - 107s 724ms/step - loss: 0.2219 - accuracy: 0.9361 - val_loss: 0.0565 - val_accuracy: 0.9880
Epoch 2/5
 34/141 [======>.......................] - ETA: 1:11 - loss: 0.0524 - accuracy: 0.9876

KeyboardInterrupt: 

In [15]:
best_model = tf.keras.models.load_model(model_path)
Yhat = crowd_label(best_model, train_X)
np.save(Yhat_path, Yhat)

In [16]:
print(f"Predict ACC: {(Yhat == train_y).sum() / len(Yhat)}")

Predict ACC: 0.9665166666666667


## Model 7: VGG16

In [26]:
model_name = "VGG16"
model_path = base_path + f"{model_name}.keras"
Yhat_path = f"./crowd labels/{model_name}_y.npy"
callbacks_list = [ ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),
                   EarlyStopping(monitor='val_accuracy', patience=10, verbose=2, mode='auto'),]

In [27]:
# Model 7: VGG16
feature_extractor = keras.applications.VGG16(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
output = final_model(inputs, feature_extractor, K) 
model = tf.keras.Model(inputs=inputs, outputs=output, name=model_name)
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics = ['accuracy'])

58900480/58889256 [==============================] - 494s 8us/step


In [28]:
history = model.fit(X0, Y0, validation_data = (X1, Y1), epochs=EPOCHS, batch_size=64, callbacks=callbacks_list,)

Epoch 1/50
141/141 [==============================] - ETA: 0s - loss: 3.4186 - accuracy: 0.4641
Epoch 1: val_accuracy improved from -inf to 0.73400, saving model to /mnt/Crowdsourcing/codes/mnist_experiment/crowd models/VGG16.keras
141/141 [==============================] - 68s 448ms/step - loss: 3.4186 - accuracy: 0.4641 - val_loss: 0.8340 - val_accuracy: 0.7340
Epoch 2/50
141/141 [==============================] - ETA: 0s - loss: 0.4626 - accuracy: 0.8493
Epoch 2: val_accuracy improved from 0.73400 to 0.88700, saving model to /mnt/Crowdsourcing/codes/mnist_experiment/crowd models/VGG16.keras
141/141 [==============================] - 60s 423ms/step - loss: 0.4626 - accuracy: 0.8493 - val_loss: 0.3249 - val_accuracy: 0.8870
Epoch 3/50
141/141 [==============================] - ETA: 0s - loss: 0.2558 - accuracy: 0.9158
Epoch 3: val_accuracy improved from 0.88700 to 0.92100, saving model to /mnt/Crowdsourcing/codes/mnist_experiment/crowd models/VGG16.keras
141/141 [=====================

In [29]:
best_model = tf.keras.models.load_model(model_path)
Yhat = crowd_label(best_model, train_X)
np.save(Yhat_path, Yhat)

## Model 8: VGG16-1

In [25]:
model_name = "VGG16-5"
model_path = base_path + f"{model_name}.keras"
Yhat_path = f"./crowd labels/{model_name}_y.npy"
callbacks_list = [ ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),
                   EarlyStopping(monitor='val_accuracy', patience=10, verbose=2, mode='auto'),]

In [26]:
# Model 8: VGG16-5
feature_extractor = keras.applications.VGG16(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
output = final_model(inputs, feature_extractor, K) 
model = tf.keras.Model(inputs=inputs, outputs=output, name=model_name)
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics = ['accuracy'])

In [27]:
history = model.fit(X0, Y0, validation_data = (X1, Y1), epochs=5, batch_size=64, callbacks=callbacks_list,)

Epoch 1/5
141/141 [==============================] - ETA: 0s - loss: 5.7201 - accuracy: 0.3543
Epoch 1: val_accuracy improved from -inf to 0.67200, saving model to /mnt/Crowdsourcing/codes/mnist_experiment/crowd models/VGG16-5.keras
141/141 [==============================] - 61s 426ms/step - loss: 5.7201 - accuracy: 0.3543 - val_loss: 0.9393 - val_accuracy: 0.6720
Epoch 2/5
141/141 [==============================] - ETA: 0s - loss: 0.7061 - accuracy: 0.7636
Epoch 2: val_accuracy improved from 0.67200 to 0.87500, saving model to /mnt/Crowdsourcing/codes/mnist_experiment/crowd models/VGG16-5.keras
141/141 [==============================] - 60s 425ms/step - loss: 0.7061 - accuracy: 0.7636 - val_loss: 0.3811 - val_accuracy: 0.8750
Epoch 3/5
141/141 [==============================] - ETA: 0s - loss: 0.3407 - accuracy: 0.8922
Epoch 3: val_accuracy improved from 0.87500 to 0.88200, saving model to /mnt/Crowdsourcing/codes/mnist_experiment/crowd models/VGG16-5.keras
141/141 [==================

In [28]:
best_model = tf.keras.models.load_model(model_path)
Yhat = crowd_label(best_model, train_X)
np.save(Yhat_path, Yhat)

In [29]:
print(f"Predict ACC: {(Yhat == train_y).sum() / len(Yhat)}")

Predict ACC: 0.9393833333333333


## Model 9: ResNet50

In [35]:
model_name = "ResNet50"
model_path = base_path + f"{model_name}.keras"
Yhat_path = f"./crowd labels/{model_name}_y.npy"
callbacks_list = [ ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),
                   EarlyStopping(monitor='val_accuracy', patience=10, verbose=2, mode='auto'),]

In [38]:
# Model 9: ResNet50
from tensorflow.keras.applications.resnet50 import ResNet50
feature_extractor = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

output = final_model(inputs, feature_extractor, K) 
model = tf.keras.Model(inputs=inputs, outputs=output, name=model_name)
callbacks_list = [ ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),
                   EarlyStopping(monitor='val_accuracy', patience=10, verbose=2, mode='auto'),]
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics = ['accuracy'])

In [39]:
history = model.fit(X0, Y0, validation_data = (X1, Y1), epochs=EPOCHS, batch_size=64, callbacks=callbacks_list,)

Epoch 1/50
141/141 [==============================] - ETA: 0s - loss: 0.2478 - accuracy: 0.9238
Epoch 1: val_accuracy improved from -inf to 0.31500, saving model to /mnt/Crowdsourcing/codes/mnist_experiment/crowd models/ResNet50.keras
141/141 [==============================] - 60s 366ms/step - loss: 0.2478 - accuracy: 0.9238 - val_loss: 10.4991 - val_accuracy: 0.3150
Epoch 2/50
141/141 [==============================] - ETA: 0s - loss: 0.0794 - accuracy: 0.9759
Epoch 2: val_accuracy improved from 0.31500 to 0.93500, saving model to /mnt/Crowdsourcing/codes/mnist_experiment/crowd models/ResNet50.keras
141/141 [==============================] - 50s 354ms/step - loss: 0.0794 - accuracy: 0.9759 - val_loss: 0.3238 - val_accuracy: 0.9350
Epoch 3/50
141/141 [==============================] - ETA: 0s - loss: 0.0444 - accuracy: 0.9871
Epoch 3: val_accuracy improved from 0.93500 to 0.97000, saving model to /mnt/Crowdsourcing/codes/mnist_experiment/crowd models/ResNet50.keras
141/141 [===========

KeyboardInterrupt: 

In [23]:
best_model = tf.keras.models.load_model(model_path)
Yhat = crowd_label(best_model, train_X)
np.save(Yhat_path, Yhat)

# Model 10: ResNet50-1

In [16]:
model_name = "ResNet50-1"
model_path = base_path + f"{model_name}.keras"
Yhat_path = f"./crowd labels/{model_name}_y.npy"
callbacks_list = [ ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),
                   EarlyStopping(monitor='val_accuracy', patience=10, verbose=2, mode='auto'),]

In [10]:
# Model 10: ResNet50-1
from tensorflow.keras.applications.resnet50 import ResNet50
feature_extractor = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

output = final_model(inputs, feature_extractor, K) 
model = tf.keras.Model(inputs=inputs, outputs=output, name=model_name)
callbacks_list = [ ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),
                   EarlyStopping(monitor='val_accuracy', patience=10, verbose=2, mode='auto'),]
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics = ['accuracy'])

2024-09-26 06:04:16.601468: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-26 06:04:17.728258: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15401 MB memory:  -> device: 0, name: Tesla P100-SXM2-16GB, pci bus id: 0000:1d:00.0, compute capability: 6.0


In [11]:
history = model.fit(X0, Y0, validation_data = (X1, Y1), epochs=1, batch_size=64, callbacks=callbacks_list,)

2024-09-26 06:06:36.010018: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


141/141 [==============================] - ETA: 0s - loss: 0.2508 - accuracy: 0.9244
Epoch 1: val_accuracy improved from -inf to 0.86800, saving model to /mnt/Crowdsourcing/codes/mnist_experiment/crowd models/ResNet50-1.keras
141/141 [==============================] - 59s 367ms/step - loss: 0.2508 - accuracy: 0.9244 - val_loss: 1.1418 - val_accuracy: 0.8680


In [17]:
best_model = tf.keras.models.load_model(model_path)
Yhat = crowd_label(best_model, train_X)
print(f"Predict ACC: {(Yhat == train_y).sum() / len(Yhat)}")
# np.save(Yhat_path, Yhat)

Predict ACC: 0.8552666666666666
